In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pydicom
import random
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.utils import check_random_state

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

import tables
from os import listdir
import glob
import tqdm
from typing import Dict
import matplotlib.pyplot as plt
%matplotlib inline


import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

from colorama import Fore, Back, Style
import pydicom
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
%matplotlib inline
import matplotlib.image as mpimg
from tabulate import tabulate
import missingno as msno 
from IPython.display import display_html
from PIL import Image
import gc
import cv2
from scipy.stats import pearsonr

import pydicom # for DICOM images
from skimage.transform import resize
import copy
import re

from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

import warnings
warnings.filterwarnings("ignore")

custom_colors = ['#74a09e','#86c1b2','#98e2c6','#f3c969','#f2a553', '#d96548', '#c14953']
sns.palplot(sns.color_palette(custom_colors))

from skimage.transform import resize
sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)

In [ ]:
def make_lungmask(img, display=False):
    row_size= img.shape[0]
    col_size = img.shape[1]
    
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    
    # Find the average pixel value near the lungs
        # to renormalize washed out images
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    
    # To improve threshold finding, I'm moving the 
    # underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate to include some of the pixels surrounding the lung.  
    # We don't want to accidentally clip the lung.

    eroded = morphology.erosion(thresh_img,np.ones([3,3]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))

    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<row_size/10*9 and B[3]-B[1]<col_size/10*9 and B[0]>row_size/5 and B[2]<col_size/5*4:
            good_labels.append(prop.label)
    mask = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0


    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation

    if (display):
        fig, ax = plt.subplots(3, 2, figsize=[12, 12])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        ax[2, 1].axis('off')
        
        plt.show()
    return mask*img

In [ ]:
patient_dir = "../input/osic-pulmonary-fibrosis-progression/train/"
patient_names = os.listdir(patient_dir)

In [ ]:
patient_names.remove('ID00011637202177653955184')
#patient_names.remove('ID00105637202208831864134')
patient_names.remove('ID00052637202186188008618')

In [ ]:
#np.shape(pixels_mean)

import h5py
#filename = '../input/ctscans64x64/my_array.h5'
filename = '../input/64-org/org_scans_64_v4.h5'


with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    data_sample = list(f[a_group_key])

In [ ]:
#np.mean(data_sample1[0]) == np.mean(data_sample1[0])

In [ ]:
pixels_mean = data_sample

In [ ]:
size_img = 64
df = pd.DataFrame([np.array(patient_names),np.array(pixels_mean).reshape(174,size_img*size_img)],['Patient','Pixels'])
#df_names = pd.DataFrame(data=np.array(patient_names).flatten())

df = df.T

In [ ]:
for i in range(len(df.Pixels)):
    df.Pixels[i] = df.Pixels[i]/max(df.Pixels[i])

In [ ]:
plt.figure()
plt.imshow(df.Pixels[0].reshape(64,64))
plt.show()

In [ ]:
plt.figure()
plt.imshow(df.Pixels[0].reshape(64,64))
plt.show()

In [ ]:
### SAVING THE CT-SCAN PIXELS AS AN HDF5 FILE
#import numpy as np
#pixels_mod
#pixels
#h5_file = tables.open_file('modified_scans_64.h5', mode='w', titel='many large arrays')
#h5_file.create_array('/', 'my_array', pixels)
#h5_file.close()

In [ ]:
patient_dir_test = "../input/osic-pulmonary-fibrosis-progression/test/"
patient_names_test  = os.listdir(patient_dir_test )

In [ ]:
%%time
#patient_dir = "../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430"

pixels1 = []
pixels_org = []

for i in range(0,len(patient_names_test)):
    datasets = []
    print(patient_names_test[i])

    # First Order the files in the dataset
    files = []
    for dcm in list(os.listdir(patient_dir_test+patient_names_test[i])):
        files.append(dcm) 
    files.sort(key=lambda f: int(re.sub('\D', '', f)))

    # Read in the Dataset
    for dcm in files:
        path = patient_dir_test+patient_names_test[i] + "/" + dcm
        datasets.append(pydicom.dcmread(path))

    imgs = []
    for data in datasets:
        img = resize(data.pixel_array, (64, 64), anti_aliasing=True)
        imgs.append(img)
        
    pixels_org.append(np.mean(imgs,axis=0))

    pixels_ = []
    for i in range(0, len(imgs)):
        resized_img = resize(datasets[i-1].pixel_array, (64, 64), anti_aliasing=True)
        img = make_lungmask(resized_img)
        pixels_.append(img)
        
    pixels1.append(np.mean(pixels_,axis=0))
    if(i%10==0):
        print(i, ' Patients are analyzed!')
        

In [ ]:
pixels_mean_test = pixels_org

In [ ]:
size_img = 64

df_test = pd.DataFrame([np.array(patient_names_test),np.array(pixels_mean_test).reshape(len(patient_names_test),size_img*size_img)],['Patient','Pixels'])
df_test = df_test.T

for i in range(len(df_test)):
    df_test.Pixels[i] = df_test.Pixels[i].flatten()
    
for i in range(len(df_test.Pixels)):
    df_test.Pixels[i] = df_test.Pixels[i]/max(df_test.Pixels[i])
    

df_test2 = pd.DataFrame([np.array(patient_names_test),np.array(pixels_mean_test).reshape(len(patient_names_test),size_img*size_img)],['Patient','Pixels'])
df_test2 = df_test2.T

for i in range(len(df_test2)):
    df_test2.Pixels[i] = df_test2.Pixels[i].flatten()
    
for i in range(len(df_test2.Pixels)):
    df_test2.Pixels[i] = df_test2.Pixels[i]/max(df_test2.Pixels[i])



In [ ]:
#df.loc[173] = ['ID00105637202208831864134'] + [df.loc[df.Patient == 'ID00067637202189903532242'].Pixels.values[0]]
df.loc[174] = ['ID00011637202177653955184'] + [df.loc[df.Patient == 'ID00342637202287526592911'].Pixels.values[0]]
df.loc[175] = ['ID00052637202186188008618'] + [df.loc[df.Patient == 'ID00165637202237320314458'].Pixels.values[0]]

In [ ]:
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(42)

In [ ]:
ROOT = "../input/osic-pulmonary-fibrosis-progression"
BATCH_SIZE= 128

In [ ]:
train = pd.read_csv(f"{ROOT}/train.csv")
train.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])
test = pd.read_csv(f"{ROOT}/test.csv")

sub = pd.read_csv(f"{ROOT}/sample_submission.csv")
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Confidence','Patient_Week']]
sub = sub.merge(test.drop('Weeks', axis=1), on="Patient")

In [ ]:
df['WHERE'] = 'train'
df_test['WHERE'] = 'val'
df_test2['WHERE'] = 'test'

In [ ]:
ct_data = []
ct_data = df.append(df_test)
ct_data = ct_data.append(df_test2)

In [ ]:
train['WHERE'] = 'train'
test['WHERE'] = 'val'
sub['WHERE'] = 'test'
data = train.append([test, sub])

data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')

In [ ]:
data['max_week'] = data['Weeks']
data.loc[data.WHERE=='test','max_week'] = np.nan
data['max_week'] = data.groupby('Patient')['max_week'].transform('max')

data['DLCO'] = data.FVC
data['min_FEV1'] = data.FVC

In [ ]:
base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC','Percent','DLCO','min_FEV1']].copy()
base.columns = ['Patient','min_FVC','min_Percent','min_DLCO','min_FEV']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)
data = data.merge(base, on='Patient', how='left')


base1 = data.loc[data.Weeks == data.max_week]
base1 = base1[['Patient','FVC','Percent']].copy()
base1.columns = ['Patient','max_FVC','max_Percent']
base1['nb'] = 1
base1['nb'] = base1.groupby('Patient')['nb'].transform('cumsum')
base1 = base1[base1.nb==1]
base1.drop('nb', axis=1, inplace=True)
data = data.merge(base1, on='Patient', how='left')

In [ ]:
data = data.merge(ct_data, on=['Patient','WHERE'])

In [ ]:
data['min_FVC1'] =  0.84 -0.3/data['min_FVC'] #0.84 -0.3/data['min_FVC']
data['FEV1'] =  0.84 -0.3/data['FVC'] #0.84 -0.3/data['min_FVC']
data['base_week'] = data['Weeks'] - data['min_week']
data['Neg_Age'] = -1/(data['Age'])

In [ ]:
COLS = ['Sex','SmokingStatus']
FE = []
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)

In [ ]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()


data['age'] = min_max_scaler.fit_transform(data['Age'].values.reshape(-1,1))
data['BASE'] = min_max_scaler.fit_transform(data['min_FVC'].values.reshape(-1,1))
data['week'] = min_max_scaler.fit_transform(data['base_week'].values.reshape(-1,1))
data['org_week'] = min_max_scaler.fit_transform(data['Weeks'].values.reshape(-1,1))
data['percent'] = min_max_scaler.fit_transform(data['Percent'].values.reshape(-1,1))
data['min_fev'] = min_max_scaler.fit_transform(data['min_FVC1'].values.reshape(-1,1))
data['neg_age'] = min_max_scaler.fit_transform(data['Neg_Age'].values.reshape(-1,1))


In [ ]:
FE = ['Ex-smoker','org_week','Never smoked','week','percent','Currently smokes','age','Female','Male','BASE','min_fev','neg_age'] 
FE2 = ['Pixels'] 

In [ ]:
train = data.loc[data.WHERE=='train']
test = data.loc[data.WHERE=='val']
sub = data.loc[data.WHERE=='test']

In [ ]:
from keras.layers import Input, Dense, InputLayer, Flatten, Reshape, BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.layers import concatenate

In [ ]:
y = train['FVC'].values
y = y.astype(float)
z1 = train[FE].values
ze1 = sub[FE].values

z = train[FE2].values
ze = sub[FE2].values

In [ ]:
final_z = []

for j in range(len(z)):
#    print(j)
    final_z.append(np.squeeze([ x for x in z[j] if not (x==i).all()]))
    
final_ze = []

for j in range(len(ze)):
#    print(j)
    final_ze.append(np.squeeze([ x for x in ze[j] if not (x==i).all()]))
    
    
z2 = np.squeeze(final_z)
ze2 = np.squeeze(final_ze)

pe = np.zeros((ze.shape[0], 3))
pred = np.zeros((z.shape[0], 3))

In [ ]:
from keras import initializers

In [ ]:
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D


In [ ]:
C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
#=============================#
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)
#============================#
def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.80]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)
#=============================#
def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss
#=================

def make_model():
    z1 = L.Input((len(FE),))
    z2 = L.Input((64*64,), name="Patient")

    init = tf.keras.initializers.VarianceScaling(scale=0.1, mode='fan_in', distribution='uniform',seed=42)

    x = L.Dense(100,kernel_initializer = init,bias_initializer='zeros', activation="relu")(z1)
    x = L.Dense(100,kernel_initializer = init,bias_initializer='zeros', activation="relu")(x)

    
    x = M.Model(inputs=z1, outputs=x)
    
    ######################
    
    y = Reshape((64*64, 1))(z2)
    
    shortcut = y
    
    y = Conv1D(kernel_initializer=init, activation='relu', 
                    padding="same", filters=4, kernel_size=8)(y)
    y = Conv1D(kernel_initializer=init, 
                    padding="same", filters=4, kernel_size=8)(y)

    y = Add()([y, shortcut])
    
    y= Activation('relu')(y)


    # Max pooling layer
    y = MaxPooling1D(pool_size=4)(y)

    # Flatten the current input for the fully-connected layers
    y = Flatten()(y)

    # Fully-connected layers
    
    y = L.Dense(256,kernel_initializer = init,bias_initializer='zeros', activation="relu")(y)
    y = L.Dense(128,kernel_initializer = init,bias_initializer='zeros', activation="relu")(y)   
    y = L.Dense(64,kernel_initializer = init,bias_initializer='zeros', activation="relu")(y)   
    y = L.Dense(32,kernel_initializer = init,bias_initializer='zeros', activation="relu")(y)   
    y = L.Dense(16,kernel_initializer = init,bias_initializer='zeros', activation="relu")(y)   
    y = L.Dense(8,kernel_initializer = init,bias_initializer='zeros', activation="relu")(y)

    y = M.Model(inputs=z2, outputs=y)


    combined = concatenate([x.output, y.output])

#    combined = L.Dense(128,kernel_initializer = init,bias_initializer='zeros', activation="relu")(combined)   
#    combined = L.Dense(64,kernel_initializer = init,bias_initializer='zeros', activation="relu")(combined)   
#    combined = L.Dense(3,kernel_initializer = init,bias_initializer='zeros', activation="relu")(combined)   

    #    combined = L.Dense(10,kernel_initializer = init,bias_initializer='zeros', activation="relu")(combined)   


    
    p1 = L.Dense(3, activation="relu", name="p1")(combined)
    p2 = L.Dense(3, activation="linear", name="p2")(combined)

    preds = L.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                     name="preds")([p1, p2])
    
    model = M.Model([z1, z2], preds, name="CNN")
    model.compile(loss=mloss(0.80), optimizer=tf.keras.optimizers.Adam(lr=0.11, beta_1=0.92, beta_2=0.998, epsilon=None, decay=0.01, amsgrad=False), metrics=[score])
    return model

In [ ]:
net = make_model()
print(net.summary())
print(net.count_params())

In [ ]:
%%time

NFOLD = 5
kf = KFold(n_splits=NFOLD)
#kf = KFold(n_splits=NFOLD, shuffle=True, random_state=42)
seed_everything(42)

pe = np.zeros((ze.shape[0], 3))
pred = np.zeros((z.shape[0], 3))

results_t = []
results_v = []

cnt = 0
for tr_idx, val_idx in kf.split(z1):
    cnt += 1
    print(f"FOLD {cnt}")
    net = make_model()
    net.fit([z1[tr_idx],z2[tr_idx]], y[tr_idx], batch_size=BATCH_SIZE, epochs=800, 
            validation_data=([z1[val_idx],z2[val_idx]], y[val_idx]), verbose=0)#,callbacks=[early_stopping]) #
    print("train", net.evaluate([z1[tr_idx],z2[tr_idx]], y[tr_idx], verbose=0, batch_size=BATCH_SIZE))
    print("val", net.evaluate([z1[val_idx],z2[val_idx]], y[val_idx], verbose=0, batch_size=BATCH_SIZE))
    results_t.append(net.evaluate([z1[tr_idx],z2[tr_idx]], y[tr_idx], verbose=0, batch_size=BATCH_SIZE))
    results_v.append(net.evaluate([z1[val_idx],z2[val_idx]], y[val_idx], verbose=0, batch_size=BATCH_SIZE))

    print("predict val...")
    pred[val_idx] = net.predict([z1[val_idx],z2[val_idx]], batch_size=BATCH_SIZE, verbose=0)
    print("predict test...")
#    net.fit([z1,z2], y, batch_size=BATCH_SIZE, epochs=800, verbose=0)#,callbacks=[early_stopping]) #
#    pe += net.predict([ze1,ze2], batch_size=BATCH_SIZE, verbose=0) / NFOLD
#net = make_model()
net.fit([z1,z2], y, batch_size=BATCH_SIZE, epochs=800, verbose=0)#,callbacks=[early_stopping]) #
pe = net.predict([ze1,ze2], batch_size=BATCH_SIZE, verbose=0) #/ NFOLD

In [ ]:
print('training loss + score: ',np.mean(results_t,axis=0))
print('validation loss + score: ',np.mean(results_v,axis=0))
print('Final loss + score: ',np.mean(results_t,axis=0)*np.mean(results_v,axis=0))

800 - +neg
training loss + score:  [39.92542267  6.49224253]

validation loss + score:  [48.23802948  6.69629803]

Final loss + score:  [1925.92371568   43.47399082]

600 - +neg

training loss + score:  [40.14292603  6.49475489]

validation loss + score:  [47.6293129   6.68711853]

Final loss + score:  [1911.97998425   43.43119575]

In [ ]:
sigma_opt = mean_absolute_error(y, pred[:, 1])
unc = pred[:,2] - pred[:, 0]
sigma_mean = np.mean(unc)
print(sigma_opt, sigma_mean)

In [ ]:
idxs = np.random.randint(0, y.shape[0], 100)
plt.plot(y[idxs],'ro', label="ground truth")
plt.plot(pred[idxs, 0],lw=2, label="q25")
plt.plot(pred[idxs, 1],lw=2, label="q50")
plt.plot(pred[idxs, 2],lw=2, label="q75")
plt.legend(loc="best")
plt.show()

In [ ]:
print(unc.min(), unc.mean(), unc.max(), (unc>=0).mean())

In [ ]:
plt.hist(unc)
plt.title("uncertainty in prediction")
plt.show()

In [ ]:
sub['FVC1'] = 0.995  * pe[:, 1]
sub['Confidence1'] = pe[:, 2] - pe[:, 0]

In [ ]:
subm = sub[['Patient_Week','FVC','Confidence','FVC1','Confidence1']].copy()

In [ ]:
subm.loc[~subm.FVC1.isnull()].head(10)

In [ ]:
subm.loc[~subm.FVC1.isnull(),'FVC'] = subm.loc[~subm.FVC1.isnull(),'FVC1']
if sigma_mean<70:
    subm['Confidence'] = sigma_opt
else:
    subm.loc[~subm.FVC1.isnull(),'Confidence'] = subm.loc[~subm.FVC1.isnull(),'Confidence1']

In [ ]:
subm.describe().T

In [ ]:
otest = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
for i in range(len(otest)):
    subm.loc[subm['Patient_Week']==otest.Patient[i]+'_'+str(otest.Weeks[i]), 'FVC'] = otest.FVC[i]
    subm.loc[subm['Patient_Week']==otest.Patient[i]+'_'+str(otest.Weeks[i]), 'Confidence'] = 0.1

In [ ]:
subm[["Patient_Week","FVC","Confidence"]].to_csv("submission.csv", index=False)

In [ ]:
good =pd.read_csv("../input/aug30th/submission-14.csv")


plt.figure(figsize=(20,10))
plt.plot(range(len(good.FVC)), good.FVC,'ro',ms=2,label='Best')
plt.plot(range(len(subm.FVC)), subm.FVC,'bo',ms=2,label='Current')

plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(range(len(good.Confidence)), good.Confidence,'ro',ms=2,label='Best')
plt.plot(range(len(subm.Confidence)), subm.Confidence,'bo',ms=2,label='Current')

plt.legend()
plt.show()